# Get the articles 
This notebook uses python3, nltk(stopwords and punkt), newspaper3k

In [ ]:
import csv
import pandas as pd
import newspaper
from newspaper import Article
import re

import csv
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [ ]:
topics = ["Events", "FiniActivity", "FundRaising", "Layoff", "MA", "Outsourcing", "PE", "PL", "Politics", "PP", "RandD"]
count_topic = 0
for topic in topics[:2]: 
    print("-"*10)
    print(count_topic)
    count_topic += 1
    output_data_file = open('output/output_data_' + topic + '.csv', 'a')
    data = pd.read_csv('data/url_list - ' + topic + '.csv', header=None)
    count = 0
    outData = []
    notFound = []
    for index, row in data.iterrows():
        print("*"*10)
        print(count)
        count += 1
        try:
            d = row[0]
            info = {}
        #------GETTING THE DATA------
            output = Article(d)
            output.download()
            output.parse()
            output = (output.text).encode('utf-8')
        #------CLEANING DATA WITH REGEX------
            if(output != ''):
                output = output.decode('utf-8') #You can get TypeError
                output = re.sub(r'\W+', ' ', output)
                output = output.replace('"', '')
                output = output.replace("'", "")
                output = output.lower()
                outData.append(output)
        #------OUTPUT DATA------
            if(output == ''):
                outData.append("NA")
                notFound.append(d)
                notFound.append(count)
            output_data_file.write((outData))
        except:
            print("ERROR!") #Occationaly link might not be working so download() would fail returning 404 error.
    output_data_file.write((outData))

Finding the frequent words in the kind of articles

In [ ]:
topics = ["Events", "FiniActivity", "FundRaising", "Layoff", "MA", "Outsourcing", "PE", "PL", "Politics", "PP", "RandD"]

In [ ]:
count_topic = 0
for topic in topics[:2]: 
    try:
        print("-"*10)
        print(count_topic)
        count_topic += 1
        input_data_file = open('output/output_data_' + topic + '.csv').read()
        input_data_file = pd.DataFrame(eval(input_data_file))
        count = 0
        outData = []
        notFound = []
        for index, row in input_data_file.iterrows():
            data = row[0]
            print("Data Type : " + str(type(data)))
            data = word_tokenize(data)
            t_data = [i for i in data if i not in stop_words and len(i) > 2]
            token_data = [i for i in t_data if i.isalpha()]
            data_pd = pd.DataFrame(token_data)
            #-----------------------------------
            print(data_pd[0].describe())
            print(set(data_pd[0].value_counts()))

            #-----------------------------------
            df = pd.DataFrame(data_pd[0].value_counts())
            print(df)
            try:
                df.to_csv("/unigramFrequency/uf_" + topic+".csv", sep=',')
            except:
                print("File/Folder not found")

            print("-"*100)

            #-----------------------------------
            bi_words = []
            for i in range(0,len(token_data)):
                bi_words.append(token_data[i-1] + " " + token_data[i])

            pd_bi_words = pd.DataFrame(bi_words)

            print(pd_bi_words[0].describe())

            print(set(pd_bi_words[0].value_counts()))

            df = pd.DataFrame(pd_bi_words[0].value_counts())
            print(df)
            try:
                df.to_csv("/bigramFrequency/bf_" + topic+".csv", sep=',')
            except:
                print("File/Folder not found")
    except:
          print("Error, may be no data")